# Multus

![](https://github.com/k8snetworkplumbingwg/multus-cni/raw/master/docs/images/Multus.png)

![](https://github.com/k8snetworkplumbingwg/multus-cni/raw/master/docs/images/multus-pod-image.svg)

Quelle: https://github.com/k8snetworkplumbingwg/multus-cni
- - -

Multus CNI ist ein Container Network Interface (CNI)-Plugin für Kubernetes, das das Anfügen mehrerer Netzwerkschnittstellen an Pods ermöglicht.


In [ ]:
# microk8s Variante
! sudo microk8s enable community
! sudo microk8s enable multus
# Multus Variante
! # kubectl apply -f https://raw.githubusercontent.com/k8snetworkplumbingwg/multus-cni/master/deployments/multus-daemonset-thick.yml

Netzwerke hinzufügen

In [ ]:
%%bash
cat <<EOF | kubectl apply -f -
---
apiVersion: "k8s.cni.cncf.io/v1"
kind: NetworkAttachmentDefinition
metadata:
  name: home-network
spec:
  config: '{
    "cniVersion": "0.3.1",
    "name": "home-network",
    "type": "bridge",
    "bridge": "br100",
    "isDefaultGateway": false,
    "forceAddress": false,
    "ipMasq": false,
    "hairpinMode": false,
    "ipam": {
      "type": "host-local",
      "subnet": "192.168.1.0/24",
      "rangeStart": "192.168.1.201",
      "rangeEnd": "192.168.1.250",
      "routes": [
        { "dst": "0.0.0.0/0" }
      ],
      "gateway": "192.168.1.1"
    }
  }'
---
apiVersion: "k8s.cni.cncf.io/v1"
kind: NetworkAttachmentDefinition
metadata:
  name: work-network
spec:
  config: '{
    "cniVersion": "0.3.1",
    "name": "work-network",
    "type": "bridge",
    "bridge": "br200",
    "isDefaultGateway": false,
    "forceAddress": false,
    "ipMasq": false,
    "hairpinMode": false,
    "ipam": {
      "type": "host-local",
      "subnet": "192.168.2.0/24",
      "rangeStart": "192.168.2.201",
      "rangeEnd": "192.168.2.250",
      "routes": [
        { "dst": "0.0.0.0/0" }
      ],
      "gateway": "192.168.2.1"
    }
  }'
EOF

In [ ]:
! kubectl describe network-attachment-definitions home-network
! kubectl describe network-attachment-definitions work-network

Erstellen eines Pods mit drei Netzwerk Schnittstellen (exkl. loopback)

In [ ]:
%%bash
cat <<EOF | kubectl apply -f -
---
apiVersion: v1
kind: Pod
metadata:
  labels:
    app: multus-alpine
  name: multus-alpine
  namespace: default
  annotations:
    k8s.v1.cni.cncf.io/networks: '[
      {
         "name" : "home-network",
         "interface": "eth1",
         "ips": ["192.168.3.205"]
      },
      {
         "name" : "work-network",
         "interface": "eth2",
         "ips": ["192.168.2.210"]
      }
    ]'
spec:
  containers:
    - name: multus-alpine
      image: alpine:latest
      command: ["sh"]
      args: ["-c", "while [ true ]; do ifconfig; sleep 3; done"]
  restartPolicy: Always
EOF

In [ ]:
! kubectl exec -it multus-alpine -- ip a

- - -

### Aufräumen

In [ ]:
! kubectl delete pod multus-alpine
! kubectl delete network-attachment-definitions home-network
! kubectl delete network-attachment-definitions work-network